Question 1 - What's the size of the converted model?

In [2]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('bees-wasps.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

2023-11-27 10:17:29.411700: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 10:17:30.948022: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-11-27 10:17:31.786163: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-11-27 10:17:31.786599: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-11-27 10:17:32.994914: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-27 10:17:32.994963: W t

In [4]:
ls -lh bees-wasps.tflite

-rw-r--r-- 1 root root 43M Nov 27 10:17 bees-wasps.tflite


Question 2: What's the output index for this model?

In [5]:
#import tensorflow.lite as tflite
import tensorflow.lite as tflite

In [6]:
interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
output_index

13

Prepare the image

In [12]:
from io import BytesIO
from urllib import request

from PIL import Image

In [13]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [16]:
url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'

img = download_image(url)

img = prepare_image(img, (150, 150))

In [17]:
import numpy as np

In [19]:
np.array(img) * (1 / 255)

array([[[0.94509804, 0.90588235, 0.85882353],
        [0.9372549 , 0.92156863, 0.97647059],
        [0.91372549, 0.89803922, 0.95686275],
        ...,
        [0.29019608, 0.33333333, 0.16470588],
        [0.34901961, 0.40784314, 0.15294118],
        [0.29803922, 0.36078431, 0.11764706]],

       [[0.94901961, 0.90980392, 0.87058824],
        [0.91764706, 0.90980392, 0.96078431],
        [0.90196078, 0.89411765, 0.94901961],
        ...,
        [0.2745098 , 0.3372549 , 0.16078431],
        [0.47058824, 0.50588235, 0.18823529],
        [0.45098039, 0.49411765, 0.18431373]],

       [[0.92941176, 0.88235294, 0.81960784],
        [0.91372549, 0.90980392, 0.96470588],
        [0.90588235, 0.89411765, 0.96078431],
        ...,
        [0.32156863, 0.37647059, 0.17647059],
        [0.50588235, 0.5254902 , 0.20392157],
        [0.43137255, 0.46666667, 0.20392157]],

       ...,

       [[0.03137255, 0.06666667, 0.04705882],
        [0.02352941, 0.08627451, 0.04705882],
        [0.08235294, 0

Question 3: After the pre-processing, what's the value in the first pixel, the R channel?

In [20]:
def preprocess_input(x):
    return x / 255.0

In [23]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [24]:
X[0, 0, 0]

array([0.94509804, 0.90588236, 0.85882354], dtype=float32)

Question 4: Now let's apply this model to this image. What's the output of the model?

In [25]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

array([[0.65898407]], dtype=float32)

Prepare the lambda code:
1. convert the notebook to python using `jupyter nbconvert --to script hw_notebook.ipynb homework.py`